## 1️⃣ Clone & Setup

In [1]:
# Clone repository
!git clone https://github.com/Usernamenisiya/thesis-cloud-rl.git
%cd thesis-cloud-rl

# Verify
!pwd
!ls -la | head -15

Cloning into 'thesis-cloud-rl'...
remote: Enumerating objects: 1684, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1684 (delta 69), reused 73 (delta 31), pack-reused 1572 (from 2)
Receiving objects: 100% (1684/1684), 653.95 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (199/199), done.
Updating files: 100% (2624/2624), done.
/content/thesis-cloud-rl
/content/thesis-cloud-rl
total 39828
drwxr-xr-x 4 root root     4096 Jan 12 05:30 .
drwxr-xr-x 1 root root     4096 Jan 12 05:29 ..
-rw-r--r-- 1 root root     1661 Jan 12 05:30 cnn_inference.py
-rw-r--r-- 1 root root    10073 Jan 12 05:30 colab_training.ipynb
drwxr-xr-x 3 root root     4096 Jan 12 05:30 data
-rw-r--r-- 1 root root     4697 Jan 12 05:30 data_download.py
-rw-r--r-- 1 root root     4735 Jan 12 05:30 evaluate_rl_model.py
drwxr-xr-x 8 root root     4096 Jan 12 05:30 .git
-rw-r--r-- 1 root root      375 Jan 12 05:30 .gitignore
-rw-r--r-- 1 root root     2167

In [2]:
# Install dependencies
!pip install -r requirements.txt
!pip install gymnasium  # Updated from deprecated gym

import torch
import stable_baselines3
import rasterio

print("✅ Dependencies installed")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 160.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.5 MB/s eta 0:00:00


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


✅ Dependencies installed
PyTorch: 2.9.0+cu126
CUDA available: True


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
# Check GPU
!nvidia-smi

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

Mon Jan 12 05:30:46 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             45W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2️⃣ Setup CloudSEN12 Real Ground Truth Data

**Using CloudSEN12 expert-labeled dataset:**
- Already downloaded: 100 patches in `Colab_Data/cloudsen12_subset/`
- Process with `cloudsen12_loader.py` to extract 10 bands
- **26M pixels** for robust evaluation

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive')

# Verify CloudSEN12 data exists
cloudsen_path = '/content/drive/MyDrive/Colab_Data/cloudsen12_subset'

if os.path.exists(cloudsen_path):
    num_patches = len([d for d in Path(cloudsen_path).iterdir() if d.is_dir()])
    print(f"✅ CloudSEN12 data found: {num_patches} patches")
    print(f"📂 Location: {cloudsen_path}")
    
    # Process CloudSEN12 data with loader (extracts 10 bands, converts masks)
    print("\n🔧 Processing CloudSEN12 patches...")
    !python cloudsen12_loader.py
else:
    print(f"❌ CloudSEN12 data not found at: {cloudsen_path}")
    print("Please run CloudSEN12 download notebook first")

Mounted at /content/drive
✅ Files copied from Google Drive
total 604M
drwxr-xr-x 3 root root 4.0K Jan 12 05:30 data
-rw------- 1 root root  29M Jan 12 05:31 ground_truth.tif
-rw------- 1 root root 575M Jan 12 05:31 sentinel2_image.tif


In [ ]:
# Verify processed CloudSEN12 data
import os
from pathlib import Path
import glob

processed_dir = 'data/cloudsen12_processed'
image_files = glob.glob(f'{processed_dir}/*_image.tif')
mask_files = glob.glob(f'{processed_dir}/*_mask.tif')

if len(image_files) > 0 and len(mask_files) > 0:
    print(f"✅ CloudSEN12 data processed successfully!")
    print(f"📊 Found {len(image_files)} image patches")
    print(f"📊 Found {len(mask_files)} mask patches")
    print("\n🎯 Ready for training with real ground truth!")
else:
    print("❌ Processed data not found")
    print("Please check cloudsen12_loader.py output for errors")

✅ All data files present!
Ready to proceed.


## 3️⃣ Check CNN Baseline

In [ ]:
# Load and test CNN baseline on CloudSEN12 patches
from cnn_inference import load_sentinel2_image, get_cloud_mask
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import rasterio
import numpy as np
import glob

print("🧠 Evaluating CNN Baseline on CloudSEN12 Real Ground Truth")
print("="*60)

# Load all processed CloudSEN12 patches
image_files = sorted(glob.glob('data/cloudsen12_processed/*_image.tif'))
mask_files = sorted(glob.glob('data/cloudsen12_processed/*_mask.tif'))

all_gt = []
all_cnn = []

print(f"Processing {len(image_files)} patches...\n")

for img_path, mask_path in zip(image_files, mask_files):  # Use ALL patches
    # Load image and get CNN prediction
    image = load_sentinel2_image(img_path)
    cnn_prob = get_cloud_mask(image)
    
    # Load real ground truth
    with rasterio.open(mask_path) as src:
        ground_truth = src.read(1)
    
    # Binary conversion
    gt_binary = (ground_truth > 0).astype(np.uint8)
    cnn_binary = (cnn_prob > 0.5).astype(np.uint8)
    
    all_gt.append(gt_binary.flatten())
    all_cnn.append(cnn_binary.flatten())

# Combine all patches
all_gt = np.concatenate(all_gt)
all_cnn = np.concatenate(all_cnn)

# Calculate metrics
accuracy = accuracy_score(all_gt, all_cnn)
precision = precision_score(all_gt, all_cnn, zero_division=0)
recall = recall_score(all_gt, all_cnn, zero_division=0)
f1 = f1_score(all_gt, all_cnn, zero_division=0)

print(f"\n📊 Evaluated on {len(image_files)} CloudSEN12 patches")
print(f"📊 Total pixels: {len(all_gt):,}")
print("\n🧠 CNN Baseline (Real Ground Truth):")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")

print(f"  F1-Score:  {f1:.4f}")print(f"📊 CNN predicted: {all_cnn.sum():,} cloud pixels ({all_cnn.mean()*100:.1f}%)")
print(f"\n📊 Ground truth: {all_gt.sum():,} cloud pixels ({all_gt.mean()*100:.1f}%)")

🧠 CNN Baseline:
  Accuracy:  0.4187
  Precision: 0.1706
  Recall:    0.6232
  F1-Score:  0.2679

📊 Ground truth: 5,144,277 cloud pixels
📊 CNN predicted: 18,786,327 cloud pixels


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4️⃣ Pull Latest Code & Train PPO

In [26]:
# Get latest code with PPO improvements
!git pull origin master
print("✅ Repository updated")

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 3), reused 5 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 2.40 KiB | 2.40 MiB/s, done.
From https://github.com/Usernamenisiya/thesis-cloud-rl
 * branch            master     -> FETCH_HEAD
   057b377..7debeb7  master     -> origin/master
Updating 057b377..7debeb7
Fast-forward
 download_cloudsen12_subset.py | 123 ++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 123 insertions(+)
 create mode 100644 download_cloudsen12_subset.py
✅ Repository updated


In [ ]:
# Optimize CNN threshold for fair baseline comparison
print("🔍 Finding optimal CNN threshold...")
!python optimize_cnn_threshold.py

In [19]:
# Run PPO training (main step - takes 1-2 hours)
print("🚀 Starting PPO training...")
print("This will take 1-2 hours with GPU")
print("="*60)

!python train_ppo.py

🚀 Starting PPO training...
This will take 1-2 hours with GPU
2026-01-12 06:28:41.461458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768199321.483381   17100 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768199321.489958   17100 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768199321.506152   17100 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768199321.506194   17100 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768199321.506

In [25]:
!python evaluate_saved_model.py models/ppo_cloud_refinement_model_20260112_062116

2026-01-12 07:23:24.410071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768202604.431626   31076 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768202604.438035   31076 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768202604.454205   31076 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768202604.454240   31076 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768202604.454243   31076 computation_placer.cc:177] computation placer alr

In [24]:
!python evaluate_saved_model.py models/ppo_cloud_refinement_model_20260112_065746

2026-01-12 07:14:10.610183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768202050.631174   28746 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768202050.637559   28746 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768202050.653443   28746 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768202050.653470   28746 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768202050.653473   28746 computation_placer.cc:177] computation placer alr

## 5️⃣ Results & Download

In [ ]:
# Display training results
import json
from pathlib import Path

results_file = Path('results/ppo_training_results.json')

if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)

    cnn = results['cnn_baseline']
    ppo = results['ppo_refined']
    imp = results['improvements']

    print("\n" + "="*60)
    print("📈 PPO TRAINING RESULTS")
    print("="*60)

    print("\n🧠 CNN Baseline:")
    print(f"  Accuracy:  {cnn['accuracy']:.4f}")
    print(f"  Precision: {cnn['precision']:.4f}")
    print(f"  Recall:    {cnn['recall']:.4f}")
    print(f"  F1-Score:  {cnn['f1_score']:.4f}")

    print("\n🤖 PPO Refined:")
    print(f"  Accuracy:  {ppo['accuracy']:.4f}")
    print(f"  Precision: {ppo['precision']:.4f}")
    print(f"  Recall:    {ppo['recall']:.4f}")
    print(f"  F1-Score:  {ppo['f1_score']:.4f}")

    print("\n🎯 Improvements:")
    print(f"  F1-Score:  {imp['f1_score_percent']:+.2f}%")
    print(f"  Accuracy:  {imp['accuracy_percent']:+.2f}%")
    print(f"  Precision: {imp['precision_delta']:+.4f}")
    print(f"  Recall:    {imp['recall_delta']:+.4f}")
    print("\n" + "="*60)
else:
    print("❌ Results file not found")
    print("Make sure PPO training completed successfully")

In [ ]:
# Save to Google Drive
import shutil
from pathlib import Path

gdrive_results = '/content/drive/MyDrive/Colab_Data/thesis_results'
Path(gdrive_results).mkdir(parents=True, exist_ok=True)

# Copy results
try:
    shutil.copy('results/ppo_training_results.json', f'{gdrive_results}/ppo_results.json')
    print("✅ Results saved to Google Drive")
except:
    print("⚠️  Could not save results to Google Drive")

# Copy model
try:
    import glob
    model_files = glob.glob('models/ppo_cloud_refinement_model*')
    for f in model_files:
        shutil.copy(f, f'{gdrive_results}/{Path(f).name}')
    print("✅ Model saved to Google Drive")
except:
    print("⚠️  Could not save model")

print(f"\n📂 Results at: {gdrive_results}")

## ✅ Summary

**Done!** Your PPO agent has been trained.

**What happened:**
1. ✅ Loaded CNN baseline performance
2. ✅ Trained PPO with balanced reward structure
3. ✅ Evaluated on test data
4. ✅ Saved results and model

**Key improvements in PPO:**
- Better exploration with entropy coefficient
- Policy gradient approach handles reward shaping better
- Larger patch size (64×64) for better context
- 100k timesteps for better convergence

**Next steps:**
1. Download results from Google Drive
2. Analyze the refined cloud mask
3. Consider hyperparameter tuning if needed

**For thesis writing:**
- See `thesis_recommendations.md` for advanced techniques
- Check `training_results.json` for detailed metrics